# 05: Score Fusion & Report Generation

This notebook demonstrates the final step: combining all three similarity signals with student-safe bias and generating academic reports.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from src.fusion import PlagiarismScorer
from src.reporting import ExplanationGenerator
from src.io import load_submissions

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Understanding Score Fusion

In [ ]:
print("🎯 Three-Signal Score Fusion")
print("=" * 70)
print("\nSimilarity Weights:")
print("  • Lexical:     15% (weak signal - supporting evidence only)")
print("  • Structural:  45% (strong signal - algorithmic structure)")
print("  • Semantic:    40% (strong signal - intent analysis)")
print("\nStudent-Safe Bias Adjustments:")
print("  ✓ Penalize if only lexical similarity is high")
print("  ✓ Boost when structural + semantic agree")
print("  ✓ Reduce score on high signal uncertainty")
print("\nSeverity Thresholds:")
print("  🚨 Severe:  ≥90% (near-exact logic copying)")
print("  ⚠️  Partial: 60-89% (core logic copied)")
print("  ✅ Clean:   <60% (original work)")
print("=" * 70)

## 2. Basic Score Fusion Example

In [ ]:
# Initialize scorer
scorer = PlagiarismScorer()

# Test with example code
code1 = '''
def factorial(n):
    if n <= 1:
        return 1
    return n * factorial(n-1)
'''

code2 = '''
def compute_fact(x):
    if x <= 1:
        return 1
    return x * compute_fact(x-1)
'''

# Compute similarity
result = scorer.compute_similarity(code1, code2, language='python', normalize=True)

print("📊 Score Fusion Example")
print("=" * 70)
print(f"\nRaw Score (before bias): {result['raw_score']:.1f}%")
print(f"Final Score (after bias): {result['final_score']:.1f}%")
print(f"\nBreakdown:")
for signal, score in result['breakdown'].items():
    print(f"  {signal.capitalize() + ':':12} {score:.1f}%")

print(f"\nStructural Method: {result['structural_method']}")
if result['structural_breakdown']:
    print(f"Structural Breakdown:")
    for method, score in result['structural_breakdown'].items():
        print(f"  {method.upper():8} {score:.1f}%")

print(f"\nSeverity: {result['severity'].upper()}")

if result['adjustments']:
    print(f"\nStudent-Safe Adjustments:")
    for adj in result['adjustments']:
        print(f"  • {adj}")

## 3. Signal Contribution Visualization

In [ ]:
# Extract signal contributions
from src.config.settings import SIMILARITY_WEIGHTS

signals = ['Lexical', 'Structural', 'Semantic']
raw_scores = [
    result['breakdown']['lexical'],
    result['breakdown']['structural'],
    result['breakdown']['semantic']
]
weights = [
    SIMILARITY_WEIGHTS['lexical'],
    SIMILARITY_WEIGHTS['structural'],
    SIMILARITY_WEIGHTS['semantic']
]
contributions = [score * weight for score, weight in zip(raw_scores, weights)]

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Raw scores
colors = ['#3498db', '#e74c3c', '#2ecc71']
bars1 = ax1.bar(signals, raw_scores, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

for bar, score in zip(bars1, raw_scores):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{score:.1f}%',
             ha='center', va='bottom', fontsize=12, fontweight='bold')

ax1.set_ylabel('Similarity Score (%)', fontsize=12, fontweight='bold')
ax1.set_title('Raw Similarity Scores', fontsize=14, fontweight='bold', pad=15)
ax1.set_ylim(0, 110)
ax1.grid(axis='y', alpha=0.3)

# Weighted contributions
bars2 = ax2.bar(signals, contributions, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

for bar, contrib, weight in zip(bars2, contributions, weights):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{contrib:.1f}%\n({weight:.0%})',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

ax2.set_ylabel('Contribution to Final Score', fontsize=12, fontweight='bold')
ax2.set_title('Weighted Contributions', fontsize=14, fontweight='bold', pad=15)
ax2.set_ylim(0, 60)
ax2.grid(axis='y', alpha=0.3)

plt.suptitle(f'Final Score = {result["final_score"]:.1f}% (after student-safe adjustments)',
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 4. Analyzing Full Dataset

In [ ]:
# Load submissions
submissions = load_submissions('../data/raw/sample_submissions.csv')

# Analyze all
print("Analyzing all submissions...\n")
results = scorer.analyze_all(submissions, normalize=True)

# Display summary
print("\n" + "=" * 70)
print("📋 Analysis Summary")
print("=" * 70)

for res in results:
    severity_emoji = {
        'severe': '🚨',
        'partial': '⚠️ ',
        'clean': '✅'
    }
    
    print(f"\n{severity_emoji[res['severity']]} {res['submission_id']}")
    print(f"   Similarity: {res['similarity_score']:.1f}%")
    print(f"   Most similar to: {res['most_similar_to']}")
    print(f"   Severity: {res['severity'].upper()}")

## 5. Severity Distribution

In [ ]:
# Count severity levels
from collections import Counter

severity_counts = Counter([r['severity'] for r in results])

# Visualize
severities = ['clean', 'partial', 'severe']
counts = [severity_counts.get(s, 0) for s in severities]
colors_sev = ['#2ecc71', '#f39c12', '#e74c3c']
labels = ['Clean\n(<60%)', 'Partial\n(60-89%)', 'Severe\n(≥90%)']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart
ax1.pie(counts, labels=labels, autopct='%1.0f%%', colors=colors_sev,
        startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
ax1.set_title('Severity Distribution', fontsize=14, fontweight='bold', pad=15)

# Bar chart
bars = ax2.bar(labels, counts, color=colors_sev, alpha=0.7, 
               edgecolor='black', linewidth=2)

for bar, count in zip(bars, counts):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{count}',
             ha='center', va='bottom', fontsize=14, fontweight='bold')

ax2.set_ylabel('Number of Submissions', fontsize=12, fontweight='bold')
ax2.set_title('Severity Counts', fontsize=14, fontweight='bold', pad=15)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nTotal submissions: {len(results)}")
for sev, count in severity_counts.items():
    percentage = count / len(results) * 100
    print(f"  {sev.capitalize():8} {count} ({percentage:.1f}%)")

## 6. Generate Academic Reports

In [ ]:
# Initialize report generator
reporter = ExplanationGenerator()

# Generate report for highest similarity submission
sorted_results = sorted(results, key=lambda x: x['similarity_score'], reverse=True)
top_result = sorted_results[0]

# Find the submission data
sub_data = next(s for s in submissions if s['submission_id'] == top_result['submission_id'])

# Generate report
report = reporter.generate_report(
    submission_id=top_result['submission_id'],
    similarity_score=top_result['similarity_score'],
    breakdown=top_result['breakdown'],
    severity=top_result['severity'],
    most_similar_to=top_result['most_similar_to'],
    code=sub_data['code'],
    adjustments=top_result.get('adjustments', [])
)

# Print formatted report
print(reporter.format_text_report(report))

## 7. Compare All Signals for Top Pair

In [ ]:
# Get top similar pair
top_pair = sorted_results[0]

# Extract breakdown
breakdown = top_pair['breakdown']

# Create radar chart
from math import pi

categories = ['Lexical', 'Structural', 'Semantic']
values = [breakdown['lexical'], breakdown['structural'], breakdown['semantic']]
values += values[:1]  # Complete the circle

angles = [n / float(len(categories)) * 2 * pi for n in range(len(categories))]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))

ax.plot(angles, values, 'o-', linewidth=2, color='#e74c3c', label='Similarity')
ax.fill(angles, values, alpha=0.25, color='#e74c3c')

# Add threshold circles
ax.plot(angles, [90]*len(angles), '--', linewidth=1, color='red', alpha=0.5, label='Severe (90%)')
ax.plot(angles, [60]*len(angles), '--', linewidth=1, color='orange', alpha=0.5, label='Partial (60%)')

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=12, fontweight='bold')
ax.set_ylim(0, 100)
ax.set_yticks([20, 40, 60, 80, 100])
ax.set_yticklabels(['20%', '40%', '60%', '80%', '100%'], fontsize=10)
ax.grid(True)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))

plt.title(f'Three-Signal Analysis\n{top_pair["submission_id"]} (Most Similar Submission)',
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print(f"\n📊 Final Score: {top_pair['similarity_score']:.1f}%")
print(f"   Verdict: {top_pair['severity'].upper()}")

## 8. Export Results

In [ ]:
import json

# Save results to JSON
output_data = {
    'analysis_summary': {
        'total_submissions': len(results),
        'severity_distribution': dict(severity_counts)
    },
    'results': results
}

with open('../data/results/plagiarism_analysis.json', 'w') as f:
    json.dump(output_data, f, indent=2)

print("✓ Results saved to data/results/plagiarism_analysis.json")

# Create DataFrame for easy viewing
df = pd.DataFrame([{
    'Submission': r['submission_id'],
    'Similarity': f"{r['similarity_score']:.1f}%",
    'Most Similar To': r['most_similar_to'],
    'Severity': r['severity'].upper(),
    'Lexical': f"{r['breakdown']['lexical']:.1f}%",
    'Structural': f"{r['breakdown']['structural']:.1f}%",
    'Semantic': f"{r['breakdown']['semantic']:.1f}%"
} for r in sorted_results])

print("\n📊 Final Results Summary:")
print(df.to_string(index=False))

## Summary

### Score Fusion Process

1. **Three Signals**: Lexical (15%) + Structural (45%) + Semantic (40%)
2. **Raw Score**: Weighted combination of all three
3. **Student-Safe Bias**: Adjustments to avoid false positives
4. **Final Score**: bias-adjusted similarity percentage
5. **Severity Classification**: Severe (≥90%) / Partial (60-89%) / Clean (<60%)

### Student-Safe Philosophy in Action

✅ **Never relies on single signal** - All three must support the verdict

✅ **Penalizes lexical-only matches** - Surface similarity isn't enough

✅ **Boosts multi-signal agreement** - Higher confidence when all agree

✅ **Reduces uncertain scores** - High variance = lower confidence

### Report Generation

✅ **Adaptive Depth**: More detailed explanations for high-similarity cases

✅ **Academic Language**: Professional, fair, evidence-based

✅ **Clear Penalties**: Specific recommendations based on severity

**Key Takeaway**: The fusion system is designed to be **conservative** - it only flags severe plagiarism when there's strong evidence across multiple independent signals. This protects students from false accusations while still catching real cheating!

**Congratulations!** You've completed the full plagiarism detection pipeline. The system is now ready for deployment and real-world testing!